## Matrix multiplication, no MR, tuples method

In [1]:
import numpy as np
from collections import defaultdict as dd
from itertools import product

For visualisation purposes lets take 2x2 matrix with integers in some range.

In [2]:
s1 = 2
s2 = 2
M = np.random.randint(low=0, high=4, size=[s1,s2]) #np.matrix([[1,0,0],[3,0,1],[2,0,0]])
N = np.random.randint(low=0, high=4, size=[s2,s1]) #np.matrix([[0,3,0],[3,0,1],[0,2,0]])

In [15]:
display(M, N, M @ N)

array([[3, 0],
       [1, 3]])

array([[3, 2],
       [3, 3]])

array([[ 9,  6],
       [12, 11]])

### Algorithm

Function "map_mat" takes matrix and returns list of tuples (index[0], index[1], value, name_of_matrix). Last parameter is only for distinction. In P=MN second matrix should be taken with parameter rev=1, which changes the indexing order.

In [3]:
def map_mat(mat, rev=0):
    s1 = np.shape(M)[0]
    s2 = np.shape(M)[1]
    m = np.ravel(mat)
    x = np.ravel(np.indices([s1,s2])[0])
    y = np.ravel(np.indices([s1,s2])[1]) 
    if not rev:
        return [(y[i],x[i],m[i], 'A') for i in range(len(m))]
    else:
        return [(x[i],y[i],m[i], 'B') for i in range(len(m))]

Change matrices to list of tuples.

In [4]:
m_ = map_mat(M)
n_ = map_mat(N, 1)

In [5]:
m_ + n_

[(0, 0, 3, 'A'),
 (1, 0, 0, 'A'),
 (0, 1, 1, 'A'),
 (1, 1, 3, 'A'),
 (0, 0, 3, 'B'),
 (0, 1, 2, 'B'),
 (1, 0, 3, 'B'),
 (1, 1, 3, 'B')]

Using DefaultDict(list) we can group those tuples with respect to the first argument. 

In [12]:
dm = dd(list)
dn = dd(list)

for j, y, v, n in m_: 
    dm[(j)].append((n, y, v))
    
for j, y, v, n in n_: 
    dn[(j)].append((n, y, v))

In [9]:
dm, dn

(defaultdict(list,
             {0: [('A', 0, 3), ('A', 1, 1)], 1: [('A', 0, 0), ('A', 1, 3)]}),
 defaultdict(list,
             {0: [('B', 0, 3), ('B', 1, 2)], 1: [('B', 0, 3), ('B', 1, 3)]}))

For each group we perform multiplying every tuple from dm with every tuple from dn in the following fashion: Combining ('A', 0, 2) and ('B', 1, 2) gives tuple ((0,1), 2*2)

In [10]:
w=[]
for group in dm:
    pairs = product(dm[group], dn[group])
    for p in pairs:
        w.append(((p[0][1], p[1][1]), p[0][2]*p[1][2]))

In [11]:
w

[((0, 0), 9),
 ((0, 1), 6),
 ((1, 0), 3),
 ((1, 1), 2),
 ((0, 0), 0),
 ((0, 1), 0),
 ((1, 0), 9),
 ((1, 1), 9)]

Tuples above only need to be grouped by first parameter and added. DefaultDict(int) can be used here to do it simply.

In [13]:
dr = dd(int)
for c, v in w: 
    dr[(c)] += v

In [14]:
dr

defaultdict(int, {(0, 0): 9, (0, 1): 6, (1, 0): 12, (1, 1): 11})

### Bigger matrix test

In [29]:
s1 = 300
s2 = 300
M = np.random.randint(low=0, high=4, size=[s1,s2])
N = np.random.randint(low=0, high=4, size=[s2,s1])

In [30]:
%%time
w=[]
m_ = map_mat(M)
n_ = map_mat(N, 1)
dm = dd(list)
dn = dd(list)
dr = dd(int)
for j, y, v, n in m_: 
    dm[(j)].append((n, y, v))
    
for j, y, v, n in n_: 
    dn[(j)].append((n, y, v))

Wall time: 221 ms


In [31]:
%%time
for group in dm:
    pairs = product(dm[group], dn[group])
    for p in pairs:
        w.append(((p[0][1], p[1][1]), p[0][2]*p[1][2]))

Wall time: 1min 31s


In [32]:
%%time
for c, v in w: 
    dr[(c)] += v

Wall time: 22.9 s


This implementation is easy to parallelize. In serialized mode it is slower than traditional methods.